In [32]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','outcome']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0.005):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    if total_rule/total_records<0.2:
        return (999,);

    avg_confidence = np.sum(rule_confidences) / count +0.1*(entropy_D-entropy_DB)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof
    # return hof



def bestIndividual(hof,df):
    k = 5
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('../data/horse/train.csv')
        df.drop('Hospital ID', axis=1, inplace=True)
        feature_columns = [col for col in df.columns if col not in ['id','outcome']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["outcome"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features


if __name__ == '__main__':
    # get dataframe path, population number and generation number from command-line argument
    # dataframePath, n_pop, n_gen = getArguments()
    n_pop=50
    n_gen=100
    # read dataframe from csv
    df = pd.read_csv('../data/horse/train.csv')
    df.drop('Hospital ID', axis=1, inplace=True)
    # encode labels column to numbers
    y = df['outcome']
    # feature_columns_con=['rectal temperature','pulse','respiratory rate',' packed cell volume','total protein']
    feature_columns = ['rectal temperature','pulse','respiratory rate',' packed cell volume','total protein','1']
    for col in feature_columns:
        df[col] = pd.qcut(df[col], q=8, labels=False, duplicates='drop')
    # get accuracy with all features
    feature_columns = [ col for col in df.columns if col not in ['id','outcome']]
    X = df[feature_columns]
    individual = [1 for i in range(len(X.columns))]
    # getFitness(df,'outcome',individual)
    print("Accuracy with all features: \t" +
          str(getFitness(df,'outcome',individual)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(df,'outcome',n_pop, n_gen)

Accuracy with all features: 	(999,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min     
0  	50    	759.286	0.129736
1  	34    	559.539	0.126818
2  	29    	199.989	0.126818
3  	29    	120.045	0.126818
4  	35    	100.052	0.126818
5  	26    	120.009	0.126818
6  	36    	20.1073	0.126818
7  	36    	0.141485	0.126818
8  	30    	60.0621 	0.126818
9  	29    	40.095  	0.126818
10 	23    	60.0716 	0.126818
11 	32    	119.992 	0.126818
12 	37    	100.022 	0.126818
13 	35    	60.0632 	0.126818
14 	31    	40.0892 	0.126818
15 	28    	60.0705 	0.126818
16 	37    	40.0856 	0.126818
17 	27    	100.017 	0.126818
18 	26    	60.067  	0.126818
19 	33    	20.1077 	0.126818
20 	31    	40.09   	0.126818
21 	31    	0.13408 	0.126818
22 	23    	40.082  	0.126818
23 	30    	119.993 	0.126818
24 	27    	20.1162 	0.126818
25 	28    	40.0852 	0.126818
26 	31    	119.993 	0.126818
27 	19    	20.1142 	0.126818
28 	36    	40.0899 	0.126818
29 	27    	20.112  	0.126818
30 	31    	20.1107 	0.126818
31 	28    	60.0613 	0.126818
32 	35    	40.0827 	0.126818
33 	29    	60.0613 	0.

In [33]:
accuracy,individual = bestIndividual(hof,df)
print(accuracy)
print(individual)


0.6733333333333332
['Age', 'pulse', 'temperature of extremities', 'mucous membranes', 'nasogastric reflux', '1', '2', '3']
